# Algorytmy tekstowe 2019/2020

# Laboratorium 3

# Autor - Łukasz Jezapkowicz

# 1. Przyjęte dane

In [3]:
def open_file(file):
    file = open(file,mode='r', encoding="utf-8")
    data = file.read()
    file.close()
    return data

In [4]:
data = []
data.append(open_file("test1.txt")) # 1kB -> 1006 B
data.append(open_file("test2.txt")) # 10kB -> 10080 B
data.append(open_file("test3.txt")) # 100kB -> 101 539 B
data.append(open_file("test4.txt")) # 1MB -> 1024kB -> 1 047 960 B

# 2. Statyczny algorytm Huffmana

Sposób przechowywania skompresowanego kodu Huffmana:  
Dany skompresowany tekst np. 101001 zapisuję do pliku binarnego.

In [5]:
from heapq import heappush,heappop
from bitarray import bitarray
from time import time
import os
from math import ceil

In [6]:
# Klasa symbolizująca węzeł w statycznym drzewie Huffmana
class StaticNode:
    def __init__(self,isLeaf=None,symbol=None,count=None,prob=None):
        self.isLeaf = isLeaf
        self.symbol = symbol
        self.count = count
        self.left = None
        self.right = None
        self.prob = prob
    
    def __lt__(self,other):
        if self.prob <= other.prob:
            return True
        else:
            return False
    
    def __eq__(self,other):
        return self.prob == other.prob

In [7]:
# Klasa symbolizująca statyczne drzewo Huffmana, lewy syn -> 0, prawy syn -> 1
class StaticHuffman:
    def __init__(self,text):
        self.root = None
        self.text = text
        self.code_length = None
        self.coded_text = None
        self.frequencies = []
        self.coded = dict()
        
    # metoda obliczająca częstość występowania oraz sortująca symbole względem tej częstości
    def calculate_frequencies(self):
        asci = [0]*256
        for c in self.text:
            asci[ord(c)] += 1
            
        for i in range(len(asci)):
            if asci[i] > 0:
                heappush(self.frequencies,(asci[i],StaticNode(True,chr(i),asci[i],asci[i]/len(self.text))))
        
    def code_letter(self,node,code):
        if node is None:
            return
        
        if node.isLeaf:
            self.coded[node.symbol]=code
            return
        
        self.code_letter(node.left,code+'0')
        self.code_letter(node.right,code+'1')
    
    # metoda kodująca znaki na odpowiednie kody binarne
    def code_letters(self):
        self.code_letter(self.root.left,'0')
        self.code_letter(self.root.right,'1')
        
    # metoda kompresująca tekst do kodu binarnego
    def compress_text(self,text):
        result = ""
        for c in text:
            result += self.coded[c]
        return result
    
    # metoda dekompresująca tekst z kodu binarnego
    def decompress_text(self,text):
        result = ""
        tmp = self.root
        for c in text:
            if tmp.isLeaf:
                result += tmp.symbol
                tmp = self.root
            if c == '0':
                tmp = tmp.left
            else:
                tmp = tmp.right
        
        # nie wolno zapomnieć o ostatnim słowie 
        return result + str(tmp.symbol)
            
    # metoda budująca drzewo Huffmana, zwraca zakodowany tekst
    def build_tree(self):
        self.calculate_frequencies()
        
        # główny krok algorytmu budowania drzewa
        while len(self.frequencies) != 1:
            min1 = heappop(self.frequencies)
            min2 = heappop(self.frequencies)
            
            new_node = StaticNode(count=min1[0]+min2[0],prob=min1[1].prob+min2[1].prob)
            new_node.left = min1[1]
            new_node.right = min2[1]
            
            heappush(self.frequencies,(min1[0]+min2[0],new_node))
        
        rt = heappop(self.frequencies)
        self.root = rt[1]
        self.code_letters()
        self.coded_text = self.compress_text(self.text)
        self.code_length = len(self.coded_text)
        return self.coded_text
    
    # metoda zwracająca współczynnik kompresji 
    def compress_coefficient(self):
        return str(100*(1 - len(self.coded_text)/(8*len(self.text)))) + "%"
    
    # metoda zapisująca skompresowany tekst do pliku
    def write_to_file(self,name):
        bits = bitarray(self.coded_text)
        with open (name,'wb') as f:
            bits.tofile(f)
    
    # metoda odczytująca tekst ze skompresowanego pliku
    def read_from_file(self,name):
        a = bitarray()
        with open(name,'rb') as f:
            a.fromfile(f)
        a = a[:self.code_length]
        txt = ""
        for bit in a:
            if bit:
                txt += "1"
            else:
                txt += "0"
        return self.decompress_text(txt)
    
    # metoda wypisujaca węzeł drzewa Huffmana
    def print_recur(self,indent,node,sym):
        if node is None:
            return
        msg = ""
        for _ in range(indent):
            msg += " "
        msg += str(sym) + " -> #" + str(node.count)
        if node.symbol is not None:
            msg += " " + str(node.symbol)
        print(msg)
        self.print_recur(indent+1,node.left,0)
        self.print_recur(indent+1,node.right,1)
        
    
    # metoda wypisująca drzewo Huffmana
    def print_tree(self):
        print("0 -> left | 1 -> right")
        print("#" + str(self.root.count))
        self.print_recur(1,self.root.left,0)
        self.print_recur(1,self.root.right,1)

# Przykład działania budowania drzewa

In [8]:
h = StaticHuffman("aaaaabbcdrr") # wzięty z wykładu
compressed = h.build_tree()
h.print_tree()
print("\nCoded letters: ")
print(h.coded)

0 -> left | 1 -> right
#11
 0 -> #5 a
 1 -> #6
  0 -> #2
   0 -> #1 c
   1 -> #1 d
  1 -> #4
   0 -> #2 b
   1 -> #2 r

Coded letters: 
{'a': '0', 'c': '100', 'd': '101', 'b': '110', 'r': '111'}


# Kompresja tekstu

In [9]:
print(h.compress_text("aaaaabbcdrr"))

00000110110100101111111


# Dekompresja tekstu

In [10]:
print(h.decompress_text("00000110110100101111111"))

aaaaabbcdrr


# Współczynnik kompresji

In [11]:
print(h.compress_coefficient())  # tekst zajmuje 11 * 8 = 88 bitów a kod binarny 23 bity -> (88-23)/88 = 73.86 %

73.86363636363636%


# Testy dla przygotowanych danych (zapisywanie oraz odczytywanie z pliku)

In [12]:
# metoda testująca zestaw danych
def test_data():
    i = 0
    for text in data:
        print("Wielkość pliku w bajtach: " + str(len(text)))
        tree = StaticHuffman(text)
        time1 = time()
        compressed = tree.build_tree()
        compress_time = time()-time1
        print("Czas kompresji: " + str(compress_time) + " sekund")
        time2 = time()
        decompressed = tree.decompress_text(tree.coded_text)
        decompress_time = time()-time2
        print("Czas dekompresji: " + str(decompress_time) + " sekund")
        
        filename = "test" + str(i+1) + "_compressed"
        i += 1
        tree.write_to_file(filename)
        decompressed_from_file = tree.read_from_file(filename)
        print("Czy skompresowany tekst oraz skompresowany tekst z pliku są sobie równe? ", end = '')
        if decompressed == decompressed_from_file:
            print("TAK")
        else:
            print("NIE")
            
        print("Czy tekst po zdekompresowaniu jest taki sam jak na początku? ", end = '')
        if decompressed == text:
            print("TAK")
        else:
            print("NIE")
            
        print("Długość oryginalnego tekstu: " + str(len(text)))
        print("Długość skompresowanego tekstu " + str(ceil(len(compressed)/8)))
        print("Teorytyczny współczynnik kompresji: " + str(round(100*(len(text)-ceil(len(compressed)/8))/len(text),4)) + "%")
        
        print("Wielkość oryginalnego pliku (w bajtach): " + str(os.path.getsize(filename[:5]+".txt")))
        print("Wielkość skompresowanego pliku (w bajtach): " + str(os.path.getsize(filename)))
        print("Rzeczywisty współczynnik kompresji: " + str(round(100*(os.path.getsize(filename[:5]+".txt")
                                                -os.path.getsize(filename))/os.path.getsize(filename[:5]+".txt"),4)) + "%\n")

In [13]:
test_data()

Wielkość pliku w bajtach: 1006
Czas kompresji: 0.0 sekund
Czas dekompresji: 0.001024007797241211 sekund
Czy skompresowany tekst oraz skompresowany tekst z pliku są sobie równe? TAK
Czy tekst po zdekompresowaniu jest taki sam jak na początku? TAK
Długość oryginalnego tekstu: 1006
Długość skompresowanego tekstu 457
Teorytyczny współczynnik kompresji: 54.5726%
Wielkość oryginalnego pliku (w bajtach): 1006
Wielkość skompresowanego pliku (w bajtach): 457
Rzeczywisty współczynnik kompresji: 54.5726%

Wielkość pliku w bajtach: 10071
Czas kompresji: 0.004987001419067383 sekund
Czas dekompresji: 0.00698089599609375 sekund
Czy skompresowany tekst oraz skompresowany tekst z pliku są sobie równe? TAK
Czy tekst po zdekompresowaniu jest taki sam jak na początku? TAK
Długość oryginalnego tekstu: 10071
Długość skompresowanego tekstu 5731
Teorytyczny współczynnik kompresji: 43.094%
Wielkość oryginalnego pliku (w bajtach): 10080
Wielkość skompresowanego pliku (w bajtach): 5731
Rzeczywisty współczynnik k

# 3. Dynamiczny algorytm Huffmanna

Sposób kodowania:  
jeżeli znak jest już w drzewie -> dodaje jego kod  
jeżeli nie znajduje -> dodaje kod znaku NYT ('#') oraz reprezentacje binarną symbolu (pełne 8 bitów)  
Pełne 8 bitów pozwala mi łatwo odkodować z pliku kolejne symbole (alternatywa - dodawanie 3 bitów
oznaczających długość kodu symbolu)  
  
Do pliku binarnego zapisuje skompresowany kod np.   011000010011000100001110010010001100011011000110010001101100 (abracadabra)

In [20]:
# Klasa symbolizująca węzeł w dynamicznym drzewie Huffmana
class DynamicNode:
    def __init__(self, parent=None, left=None, right=None, weight=0, symbol=None):
        self.parent = parent
        self.symbol = symbol
        self.left = left
        self.right = right
        self.weight = weight

In [105]:
# Klasa symbolizująca dynamiczne drzewo Huffmana
class DynamicHuffman:
    def __init__(self,print_time = False):
        self.nyt = DynamicNode(symbol='#')
        self.root = self.nyt
        self.used = [None]*256
        self.nodes = [self.nyt]
        self.print_time = print_time
        self.time = 0
    
    # metoda zwracająca kod dla znaku
    def get_symbol_code(self,c,c_node,c_code = ''):
        # liść
        if c_node.left is None and c_node.right is None:
            return c_code if c_node.symbol == c else ''
        
        else:
            code = ''
            if c_node.left is not None:
                code = self.get_symbol_code(c,c_node.left,c_code + '0')
            if not code and c_node.right is not None:
                code = self.get_symbol_code(c,c_node.right,c_code + '1')
            return code
    
    # metoda naprawiająca drzewo (swapująca węzły)
    def update(self,node1,node2):
        # biorę odpowiednie indeksy i zamieniam węzły miejscami w liście
        index1, index2 = self.nodes.index(node1), self.nodes.index(node2)
        self.nodes[index1], self.nodes[index2] = self.nodes[index2], self.nodes[index1]

        # naprawiam odpowiednie łączenia
        tmp = node1.parent
        node1.parent = node2.parent
        node2.parent = tmp
        
        if node1.parent.left is node2:
            node1.parent.left = node1
        else:
            node1.parent.right = node1
        
        if node2.parent.left is node1:
            node2.parent.left = node2
        else:
            node2.parent.right = node2
    
    # metoda znajdująca najdalszy węzeł w liście o danej wadze
    def find_furthest(self,weight):
        for n in reversed(self.nodes):
            if n.weight == weight:
                return n
    
    # metoda wstawiająca kolejny znak do drzewa Huffmana
    def insert(self,c):
        # biorę odpowiedni węzeł
        c_node = self.used[ord(c)]
        
        # jeśli nie istnieje to go tworzę
        if c_node is None:
            # tworze odpowiednie struktury
            new_node = DynamicNode(weight = 1, symbol = c)
            new_internal = DynamicNode(left = self.nyt,right = new_node, parent = self.nyt.parent,
                                weight = 1,symbol='')
            new_node.parent = new_internal
            self.nyt.parent = new_internal
            
            # podpinam nowy węzeł zewnętrzny do jego ojca albo ustawiam jako root
            if new_internal.parent is not None:
                new_internal.parent.left = new_internal
            else:
                self.root = new_internal

            # dodaje nowe węzły na początek listy węzłów
            self.nodes.insert(0, new_internal)
            self.nodes.insert(0, new_node)
            self.used[ord(c)] = new_node
            # jako odpowiedni węzeł biorę ojca ojca nowo wstawionego węzła - dopiero w nim
            # mogło się coś zmienić
            c_node = new_internal.parent
        
        # aktualizuję wagi i poprawiam drzewo w razie potrzeby
        while c_node is not None:            
            # znajduję węzeł o tej samej wadze, który jest najdalej na liście
            furthest = self.find_furthest(c_node.weight)
            
            # jeżeli nasz węzeł nie jest najdalszym, lub nie jest z nim 
            # połączony relacją ojciec-syn naprawiamy drzewo
            if (c_node is not furthest and 
                furthest is not c_node.parent):
                self.update(c_node, furthest)
            
            c_node.weight += 1
            c_node = c_node.parent
    
    # metoda budująca dynamiczne drzewo Huffmana
    def build_tree(self,text):
        coded = ''
        
        for c in text:
            # dodajemy aktualny kod znaku
            time1 = time()
            if self.used[ord(c)] is not None:
                tmp1 = self.get_symbol_code(c, self.root)
                coded += tmp1
            else:
                tmp2 = self.get_symbol_code('#',self.root)
                coded += tmp2
                # dodaje binarną reprezentację znaku i opcjonalnie wypełniam 0 do długości 8 (bajt)
                # wypełnianie oczywiście od najbardziej znaczących bitów
                coded += bin(ord(c))[2:].zfill(8)
            self.time += time()-time1
                
            self.insert(c)
        
        self.coded = coded
        self.code_length = len(coded)
        if self.print_time:
            print("Czas szukania kodów symbolów = " + str(self.time) + " sekund")
        
        return coded
    
    # metoda odkodowująca zakodowany tekst
    def decode_text(self, text):
        uncoded = ''
        
        # odkodowuję pierwszy znak
        c_symbol = chr(int(text[:8],2))
        uncoded += c_symbol
        self.insert(c_symbol)
        c_node = self.root
        
        i = 8
        # odkodowuję resztę znaków
        while i < len(text):
            c_node = c_node.left if text[i] == '0' else c_node.right
            c_symbol = c_node.symbol
            
            # jeżeli znaleźliśmy symbol
            if c_symbol:
                if c_symbol == '#':
                    c_symbol = chr(int(text[i+1:i+9],2))
                    i += 8
                
                uncoded += c_symbol
                self.insert(c_symbol)
                c_node = self.root
            
            i += 1
        
        return uncoded
    
    # metoda zapisująca skompresowany tekst do pliku
    def write_to_file(self,name,text):
        coded = self.build_tree(text)
        bits = bitarray(coded)
        with open (name,'wb') as f:
            bits.tofile(f)
        return len(coded)
            
    # metoda odczytująca tekst ze skompresowanego pliku
    def read_from_file(self,name,length):
        a = bitarray()
        with open(name,'rb') as f:
            a.fromfile(f)
        a = a[:length]
        txt = ""
        for bit in a:
            if bit:
                txt += "1"
            else:
                txt += "0"
        return self.decode_text(txt)

# Kompresja tekstu

In [106]:
print(DynamicHuffman().build_tree("abracadabra"))

011000010011000100001110010010001100011011000110010001101100


# Dekompresja tekstu

In [107]:
print(DynamicHuffman().decode_text("011000010011000100001110010010001100011011000110010001101100"))

abracadabra


# Współczynnik kompresji

In [108]:
print(str(100*(len("abracadabra")*8-len(DynamicHuffman().build_tree("abracadabra")))/
      (8*len("abracadabra"))) + '%')
print("Jak widać, o wiele gorzej w przypadku tego krótkiego słowa ")

31.818181818181817%
Jak widać, o wiele gorzej w przypadku tego krótkiego słowa 


# Testy dla przygotowanych danych (zapisywanie oraz odczytywanie z pliku)

In [109]:
# metoda testująca zestaw danych
def test_data_2():
    i = 0
    for text in data:
        print("Wielkość pliku w bajtach: " + str(len(text)))
        time1 = time()
        compressed = DynamicHuffman(True).build_tree(text)
        compress_time = time()-time1
        print("Czas kompresji: " + str(compress_time) + " sekund")
        time2 = time()
        decompressed = DynamicHuffman().decode_text(compressed)
        decompress_time = time()-time2
        print("Czas dekompresji: " + str(decompress_time) + " sekund")
        
        filename = "test" + str(i+1) + "_dynamic_compressed"
        i += 1
        length = DynamicHuffman().write_to_file(filename,text)
        decompressed_from_file = DynamicHuffman().read_from_file(filename,length)
        print("Czy skompresowany tekst oraz skompresowany tekst z pliku są sobie równe? ", end = '')
        if decompressed == decompressed_from_file:
            print("TAK")
        else:
            print("NIE")
                    
        print("Czy tekst po zdekompresowaniu jest taki sam jak na początku? ", end = '')
        if decompressed == text:
            print("TAK")
        else:
            print("NIE")
        
        print("Długość oryginalnego tekstu: " + str(len(text)))
        print("Długość skompresowanego tekstu " + str(ceil(len(compressed)/8)))
        print("Teorytyczny współczynnik kompresji: " + str(round(100*(len(text)-ceil(len(compressed)/8))/len(text),4)) + "%")
        
        print("Wielkość oryginalnego pliku (w bajtach): " + str(os.path.getsize(filename[:5]+".txt")))
        print("Wielkość skompresowanego pliku (w bajtach): " + str(os.path.getsize(filename)))
        print("Rzeczywisty współczynnik kompresji: " + str(round(100*(os.path.getsize(filename[:5]+".txt")
                                                -os.path.getsize(filename))/os.path.getsize(filename[:5]+".txt"),4)) + "%\n")

In [103]:
test_data_2()

Wielkość pliku w bajtach: 1006
Czas szukania kodów symbolów = 0.007980823516845703 sekund
Czas kompresji: 0.012995481491088867 sekund
Czas dekompresji: 0.007010698318481445 sekund
Czy skompresowany tekst oraz skompresowany tekst z pliku są sobie równe? TAK
Czy tekst po zdekompresowaniu jest taki sam jak na początku? TAK
Długość oryginalnego tekstu: 1006
Długość skompresowanego tekstu 489
Teorytyczny współczynnik kompresji: 51.3917%
Wielkość oryginalnego pliku (w bajtach): 1006
Wielkość skompresowanego pliku (w bajtach): 489
Rzeczywisty współczynnik kompresji: 51.3917%

Wielkość pliku w bajtach: 10071
Czas szukania kodów symbolów = 0.2882812023162842 sekund
Czas kompresji: 0.3600635528564453 sekund
Czas dekompresji: 0.08585429191589355 sekund
Czy skompresowany tekst oraz skompresowany tekst z pliku są sobie równe? TAK
Czy tekst po zdekompresowaniu jest taki sam jak na początku? TAK
Długość oryginalnego tekstu: 10071
Długość skompresowanego tekstu 5809
Teorytyczny współczynnik kompresji:

# 4. Porównanie współczynników kompresji 

## Statyczny

Test_1 -> 54.5726 %  
Test_2 -> 43.1448 %  
Test_3 -> 46.2630 %  
Test_4 -> 45.7380 %

## Dynamiczny

Test_1 -> 51.3917 %  
Test_2 -> 42.3710 %  
Test_3 -> 46.1734 %  
Test_4 -> 45.7249 %

# 5. Wnioski (co jest nie tak z dynamicznym algorytmem) 

Dynamiczny algorytm zgodnie z moimi oczekiwaniami na plikach sensownej długości potrafił osiągnąć współczynnik kompresji zbliżony do algorytmu statycznego.  
Jednak algorytm okazał się o wiele wolniejszy, dlaczego?  
Pierwszym czynnikiem jest liniowe przeszukiwanie tablicy węzłów, które  
stanowiło ok 15 % czasu trwania algorytmu (jest to spokojnie do naprawy zmieniając strukturę przechowywania węzłów). Skąd to 85%?  
Drugi czynnik okazał się dla mnie niezrozumiały. Otóż około 80% czasu trwania
algorytmu zajmuje procedura rekurencyjna znajdujaca kod symbolu. Procedura ta ma optymistyczny rząd złożoności O(logn) zaś pesymistyczny O(n). Pesymistycznie powinna więc być w okolicach czasu tych 15% jak przeszukiwanie tablicy a osiąga
gigantyczne 80%. Nie jestem w stanie zrozumieć dlaczego tak się dzieje. Gdyby Pan Doktor wiedział co mogło spowodować ten problem, proszę o feedback.  
  
P.S wypisałem dodatkowo pełen czas trwania procedury rekurencyjnej żeby widzieć że osiąga 80%